### Linear Regression example 2

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

####  load data

In [4]:
path = '/home/jovyan/work/original/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Linear_Regression/Ecommerce_Customers.csv'
data = spark.read.csv(path, inferSchema=True, header=True)

In [5]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [6]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


#### preapare dataset for machine learning

In [7]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [8]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [9]:
assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App', 'Time on Website', 
                                       'Length of Membership'],
                            outputCol='features')

In [10]:
output = assembler.transform(data)

In [11]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [12]:
final_data = output.select('features', 'Yearly Amount Spent')

In [13]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [14]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                360|
|   mean|  498.5382060152329|
| stddev|  79.70930006971061|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+



#### Create and train model

In [15]:
lr = LinearRegression(labelCol="Yearly Amount Spent")

In [16]:
lr_model = lr.fit(train_data)

In [17]:
test_results = lr_model.evaluate(test_data)

In [18]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| 11.870167597222235|
|  7.500678681489717|
| 10.963688358199988|
| -3.151519319004592|
| 19.707299163049413|
| -4.742984212086412|
| -4.855018425675439|
| -8.198247786105583|
| 18.712413727504327|
|-0.2526911328703818|
| -6.557634720435203|
| -5.738503079162172|
|-1.7000295079225793|
| 1.7238376311213415|
|  8.752262340210848|
|-1.8507715619325609|
|-1.5523428061602544|
| -9.827902535571525|
| -16.72608671164909|
|  -17.5622448913241|
+-------------------+
only showing top 20 rows



In [19]:
test_results.rootMeanSquaredError

9.633627415646362

In [20]:
test_results.r2

0.9848464598340397

In [21]:
unlabeled_data = test_data.select('features')

In [22]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.9716756438877...|
|[31.1695067987115...|
|[31.2681042107507...|
|[31.3123495994443...|
|[31.5147378578019...|
|[31.5257524169682...|
|[31.5261978982398...|
|[31.6005122003032...|
|[31.6610498227460...|
|[31.7207699002873...|
|[31.7242025238451...|
|[31.8186165667690...|
|[31.8293464559211...|
|[31.8512531286083...|
|[31.8530748017465...|
|[31.8627411090001...|
|[31.8648325480987...|
|[31.9048571310136...|
|[31.9563005605233...|
+--------------------+
only showing top 20 rows



In [23]:
predictions = lr_model.transform(unlabeled_data)

In [24]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...|396.77018347540525|
|[30.9716756438877...|  487.137931075403|
|[31.1695067987115...| 416.3928424440928|
|[31.2681042107507...| 426.6220524928285|
|[31.3123495994443...| 443.8841188648912|
|[31.5147378578019...| 494.5554722085478|
|[31.5257524169682...| 448.8206452355573|
|[31.5261978982398...| 417.2927739784434|
|[31.6005122003032...| 460.4604377635926|
|[31.6610498227460...| 416.6110447127712|
|[31.7207699002873...| 545.3325681984581|
|[31.7242025238451...|509.12639036712267|
|[31.8186165667690...|448.11870287805823|
|[31.8293464559211...|383.42850035685365|
|[31.8512531286083...|464.23998432658755|
|[31.8530748017465...|461.13589502428454|
|[31.8627411090001...|  557.850483980207|
|[31.8648325480987...| 449.7191830123852|
|[31.9048571310136...| 490.6759441344652|
|[31.9563005605233...| 564.6881766385229|
+--------------------+------------